In [1]:
import pandas as pd
import numpy as np

# Loading data from my drive and filter rows where kind is not null
data = pd.read_csv('/content/drive/MyDrive/Data Sheet.csv') #Code created in colab. Kept data sheet in drive for ease of use.
data = data[data['kind'].notnull()]

In [2]:
# Converting 'created_at' to datetime and extract sortable year-month format
data['created_at'] = pd.to_datetime(data['created_at'])
data['Year_Month'] = data['created_at'].dt.strftime('%Y-%m')

# Calculating the delivery time and group by kind/products & year/Month
data['updated_at'] = pd.to_datetime(data['updated_at'], errors='coerce')
data['delivery_time'] = (data['updated_at'] - data['created_at']).dt.days

# Group by product and Year/Month to get average and tp90 delivery time
stats = data.groupby(['kind', 'Year_Month']).agg({'delivery_time': ['mean', lambda x: np.quantile(x, 0.9)]})

# Count number of customers with at least 1 delivery per month
num_customers = data.groupby(['Year_Month']).agg({'booking_id': 'nunique'}).values.flatten()
num_customers2 = num_customers
num_cus = np.concatenate((num_customers, num_customers2)) # The data array shape was 9, i wanted to repeat the numbers twice for average & tp90 delivery time. Hence i used .concatenate to tackle that issue. to match table columns.


In [4]:
# formatted output DataFrame
output_data = pd.DataFrame({
    'Product Name': stats.index.get_level_values('kind'),
    'Year_Month': stats.index.get_level_values('Year_Month'),
    'Average Delivery Time (days)': stats['delivery_time']['mean'].values,
    'tp90 Delivery Time (days)': stats['delivery_time']['<lambda_0>'].values
})


In [5]:
# Pivot the DataFrame & add a row for number of customers per month
output_data_pivot = output_data.pivot_table(index='Product Name', columns='Year_Month', fill_value=np.nan)
output_data_pivot.loc['No. of customers with atleast 1 delivery'] = num_cus

In [6]:
# Sorting of columns in ascending order
output_data_pivot = output_data_pivot.reindex(sorted(output_data_pivot.columns), axis=1)

# Save to Excel file
output_data_pivot.to_excel('Output Result.xlsx')

In [7]:
# Display first few rows to check data
print(output_data_pivot.head())

                          Average Delivery Time (days)              \
Year_Month                                     2023-06     2023-07   
Product Name                                                         
air_freight-tracking                        182.206897  152.040000   
bonds-tracking                              102.000000   76.000000   
dd-tracking                                  72.222222   58.928571   
fcl-tracking                                 75.666667   78.750000   
fcl-transloading-tracking                          NaN         NaN   

                                                                        \
Year_Month                    2023-08    2023-09    2023-10    2023-11   
Product Name                                                             
air_freight-tracking       118.989011  97.768116  65.088435  65.254777   
bonds-tracking              32.285714  26.857143   7.250000  19.000000   
dd-tracking                 62.750000  69.300000  67.181818  54.56250